## Initial Exploratory and Data Cleaning

In [130]:
import re
import string
import pickle
import numpy as np
import pandas as pd
pd.set_option('max_colwidth',150)
from prettytable import PrettyTable # pip install prettytable
from sklearn.feature_extraction.text import CountVectorizer

In [103]:
path = '/Users/mouhamethtakhafaye/Desktop/behavox_assignment/src/data/01_raw.pkl' 

In [104]:
raw_df = pd.read_pickle(path)

In [105]:
raw_df.dtypes

Message-ID                   object
Date                         object
From                         object
To                           object
Subject                      object
Mime-Version                 object
Content-Type                 object
Content-Transfer-Encoding    object
X-From                       object
X-To                         object
X-cc                         object
X-bcc                        object
X-Folder                     object
X-Origin                     object
X-FileName                   object
Body                         object
Cc                           object
Bcc                          object
dtype: object

In [106]:
raw_df.index = raw_df['Date'].apply(pd.to_datetime)

# Remove non-essential columns
cols_to_keep = ['From', 'To', 'Cc', 'Bcc', 'Subject', 'Body']
raw_df = raw_df[cols_to_keep]

In [107]:
raw_df.head()

,From,To,Cc,Bcc,Subject,Body
Date,,,,,,
2001-12-07 10:06:42-08:00,heather.dunton@enron.com,k..allen@enron.com,NaN,NaN,RE: West Position,"Please let me know if you still need Curve Shift. Thanks, Heather -----Original Message----- From: Allen, Phillip K. Sent: Friday, December 0..."
2001-12-30 22:49:42-08:00,anchordesk_daily@anchordesk.zdlists.com,pallen@enron.com,NaN,NaN,ANCHORDESK: Hope ahead: What I learned from 2001's tragedies,"_____________________DAVID COURSEY_____________________ HOPE AHEAD: WHAT I LEARNED FROM 2001'S TRAGEDIES As years go, 2001 sucked. But adver..."
2001-12-30 23:42:30-08:00,subscriptions@intelligencepress.com,pallen@enron.com,NaN,NaN,"NGI Publications - Monday, December 31st 2001","Dear phillip, This e-mail is automated notification of the availability of your current Natural Gas Intelligence Newsletter(s). Please use your ..."
2001-12-31 02:24:51-08:00,prizemachine@feedback.iwon.com,pallen@enron.com,NaN,NaN,"Click. Spin. Chances to Win up to $10,000!","[IMAGE] [IMAGE] [IMAGE] [IMAGE] $ 2,500 [IMAGE] [IMAGE] [IMAGE] Dear Phillip, You've got to spin to win! Play now! Spin the iWon..."
2001-12-31 10:53:43-08:00,louise.kitchen@enron.com,"wes.colwell@enron.com, georgeanne.hodges@enron.com, rob.milnthorp@enron.com,\n\tjohn.zufferli@enron.com, peggy.hedstrom@enron.com,\n\tthomas.myers...",john.lavorato@enron.com,john.lavorato@enron.com,NETCO,The New Year has arrived and we really to finalize a lot of the work with regards to moving into NETCO. Obviously we still do not have a deal but...


In [108]:
raw_df.shape

(68, 6)

## Analyzing Patterns in Sender/Recipient Communications

In [109]:
senders = raw_df['From'].unique()
receivers = raw_df['To'].unique()
cc_receivers = raw_df['Cc'].unique()
bcc_receivers = raw_df['Bcc'].unique()

print('Num Senders:', len(senders))
print('Num Receivers:', len(receivers))
print('Num CC Receivers:', len(cc_receivers))
print('Num BCC Receivers:', len(bcc_receivers))

Num Senders: 44
Num Receivers: 18
Num CC Receivers: 11
Num BCC Receivers: 11


In [110]:
senders = set(senders)
receivers = set(receivers)
cc_receivers = set(cc_receivers)
bcc_receivers = set(bcc_receivers)

# Find the number of senders who were also direct receivers

senders_intersect_receivers = senders.intersection(receivers)

# Find the senders that didn't receive any messages

senders_diff_receivers = senders.difference(receivers)
                                           
# Find the receivers that didn't send any messages

receivers_diff_senders = receivers.difference(senders)

# Find the senders who were any kind of receiver by
# first computing the union of all types of receivers

all_receivers = receivers.union(cc_receivers, bcc_receivers)
senders_all_receivers = senders.intersection(all_receivers)

print("Num senders in common with receivers:", len(senders_intersect_receivers))
print("Num senders who didn't receive:", len(senders_diff_receivers))
print("Num receivers who didn't send:", len(receivers_diff_senders))
print("Num senders in common with *all* receivers:", len(senders_all_receivers))

Num senders in common with receivers: 3
Num senders who didn't receive: 41
Num receivers who didn't send: 15
Num senders in common with *all* receivers: 4


## Who is Sending and Receiving the Most Email?

In [111]:
top_senders = raw_df.groupby('From')
top_receivers = raw_df.groupby('To')

top_senders = top_senders.count()['To']
top_receivers = top_receivers.count()['From']

# Get the ordered indices of the top senders and receivers in descending order
top_snd_ord = np.argsort(top_senders)[::-1]
top_rcv_ord = np.argsort(top_receivers)[::-1]

top_senders = top_senders[top_snd_ord]
top_receivers = top_receivers[top_rcv_ord]

In [112]:
top10 = top_senders[:10]
pt = PrettyTable(field_names=['Rank', 'Sender', 'Messages Sent'])
pt.align['Messages Sent'] = 'r'
[ pt.add_row([i+1, email, vol]) for i, email, vol in zip(range(10), top10.index.values, top10.values)]

print(pt)

+------+-------------------------------------+---------------+
| Rank |                Sender               | Messages Sent |
+------+-------------------------------------+---------------+
|  1   |      arsystem@mailman.enron.com     |             5 |
|  2   | subscriptions@intelligencepress.com |             4 |
|  3   |       kirk.mcdaniel@enron.com       |             3 |
|  4   |      mery.l.brown@accenture.com     |             3 |
|  5   |         wise.counsel@lpl.com        |             2 |
|  6   |        msimpkins@winstead.com       |             2 |
|  7   |        gthorse@about-cis.com        |             2 |
|  8   |    hunter.williams@grandecom.com    |             2 |
|  9   |        james.bruce@enron.com        |             2 |
|  10  |        webmaster@earnings.com       |             2 |
+------+-------------------------------------+---------------+


In [123]:
top10 = top_receivers[:10]
pt = PrettyTable(field_names=['Rank', 'Receiver', 'Messages Received'])
pt.align['Messages Sent'] = 'r'
[ pt.add_row([i+1, email, vol]) for i, email, vol in zip(range(10), top10.index.values, top10.values)]

#print(pt)

[None, None, None, None, None, None, None, None, None, None]

In [115]:
df = raw_df.Body.copy()

In [116]:
df

Date
2001-12-07 10:06:42-08:00    Please let me know if you still need Curve Shift.  Thanks, Heather  -----Original Message----- From:  Allen, Phillip K.   Sent: Friday, December 0...
2001-12-30 22:49:42-08:00    _____________________DAVID COURSEY_____________________  HOPE AHEAD: WHAT I LEARNED FROM 2001'S TRAGEDIES      As years go, 2001 sucked. But adver...
2001-12-30 23:42:30-08:00    Dear phillip,   This e-mail is automated notification of the availability of your current Natural Gas Intelligence Newsletter(s). Please use your ...
2001-12-31 02:24:51-08:00    [IMAGE] [IMAGE]   [IMAGE]   [IMAGE] $ 2,500 [IMAGE]   [IMAGE]  [IMAGE]      Dear  Phillip,  You've got to spin to win! Play now!     Spin the iWon...
2001-12-31 10:53:43-08:00    The New Year has arrived and we really to finalize a lot of the work with regards to moving into NETCO.  Obviously we still do not have a deal but...
2001-12-31 17:18:31-08:00    This request has been pending your approval for  59 days.  Please click

## Apply a first round of text cleaning techniques


In [117]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Remove square brackets
round1 = lambda x: clean_text_round1(x)

In [118]:
data_clean = pd.DataFrame(df.apply(round1))
data_clean.sample(10)

,Body
Date,
2001-11-27 08:02:04-08:00,michelle here are my very minor comments however we still need to wait on any additions based on meeting with smes today one concern is the firing...
2001-12-10 15:31:51-08:00,phillip my interpretation of this is that we made total half from new deals and the other half from reserve releases and when you back out the pr...
2001-10-29 14:03:58-08:00,amazoncom delivers home garden home garde n editor teri kieffer october search booksrare used books kids booksspanis...
2001-12-10 14:59:55-08:00,attached is the information you have requested thanks brad jones
2001-11-16 10:07:13-08:00,wow this looks complicated let me get some help on this and get back to you original message from phillip k november am greg lavorat...
2001-12-31 02:24:51-08:00,dear phillip youve got to spin to win play now spin the iwon prize machine for chances to win the progressive jackpo...
2001-10-29 16:22:13-08:00,this request has been pending your approval for days please click to review and act upon this request request id request creat...
2001-11-27 15:19:15-08:00,this email is not sent unsolicited this is an mailing this message is sent to subscribers only the email subscription address is pallenenr...
2001-10-25 12:04:35-07:00,phillip pursuant to your request please see the attached thanks renee


##  Apply a second round of cleaning

In [119]:
def clean_text_round2(text):
    '''Get rid of some additional quotation marks and newline text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [122]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.Body.apply(round2))
data_clean.sample(10)

,Body
Date,
2001-11-27 16:50:21-08:00,hello integrity realty services has the following property available for sale north rr austin texas a one story retail strip center with an ...
2001-10-29 09:16:09-08:00,sheri and i would like to discuss the practice questions and graphic ideas with you for the the knowledge system we wanted to get some feedback f...
2001-12-31 22:54:34-08:00,david coursey in review not perfect but it sure beat welcome to my year in review column which i feel very safe in asserting you...
2001-10-10 09:03:56-07:00,phillip there are a number of alternative systems that will allow the same level of energy efficiency i would wait a bit for winks bid though you ...
2001-11-27 08:10:09-08:00,team fyi sheri the smes that have already committed to being on film need to be keep in the lope regarding the timeline also check with these s...
2001-10-29 17:35:18-08:00,this request has been pending your approval for days please click to review and act upon this request request id request creat...
2001-11-27 17:22:24-08:00,this request has been pending your approval for days please click to review and act upon this request request id request creat...
2001-09-11 10:12:32-07:00,gregphillip attached is the grande communications service agreement the business points can be found in exhibit c i can get the nondisturbance ...
2001-11-27 15:19:15-08:00,this email is not sent unsolicited this is an mailing this message is sent to subscribers only the email subscription address is pallenenr...


## Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [127]:
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.Body) # fit count vectorizor to our CLEAN transcript data

# Convert it to an array and label all the columns
# Can use this part for future projects
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index

# Document-Term matrix
data_dtm.sample(10)

,able,abn,accelerated,accelerating,accenture,acceptance,accepted,access,accessing,accessories,...,youre,youve,yr,ys,yuletide,zd,zdnet,zero,zipper,zone
Date,,,,,,,,,,,,,,,,,,,,,
2001-11-26 15:33:45-08:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-10-25 13:24:44-07:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-11-27 15:19:15-08:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-11-16 12:22:12-08:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-10-29 14:03:58-08:00,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,1,0,0,0,0,0
2001-10-25 12:04:35-07:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-10-29 17:35:18-08:00,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-11-26 08:31:11-08:00,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
# Let's pickle it for later use
data_dtm.to_pickle("02.dtm.pkl")

In [131]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('03.data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))